<a href="https://colab.research.google.com/github/EmmaJaneShaw/macquarie/blob/main/colab_default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [11]:
!pip install pyjanitor

In [12]:
import anvil.server
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O, data manipulation 
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from pandas import set_option
plt.style.use('ggplot') # nice plots
import janitor as jn

from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.linear_model import LogisticRegression # to apply the Logistic regression
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold # for cross validation
from sklearn.model_selection import GridSearchCV # for tuning parameter
from sklearn.model_selection import RandomizedSearchCV  # Randomized search on hyper parameters.
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os
#print(os.listdir("../input"))

In [3]:
data = pd.read_csv('/content/drive/MyDrive/macquarie_data/credit_card.csv')
data.sample(5)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,SK_ID_CURR,SRC_VAR_1,SRC_VAR_4,WEEKDAY_APPR_PROCESS_START
186719,20336.0,395766,283500.0,67500,0.0,0.0,0.0,0.0,0.0,3.0,...,0,0,0,0.002,3,3,334284,-1.545,0.212,TUESDAY
53199,33737.0,948582,679500.0,360000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,0.073,1,1,165774,-0.431,0.605,WEDNESDAY
277257,68360.0,701730,675000.0,315000,0.0,0.0,0.0,0.0,0.0,7.0,...,1,1,1,0.015,2,2,448140,-0.840,0.419,MONDAY
21688,35325.0,1125000,1125000.0,262062,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.031,2,2,126120,-0.382,0.631,TUESDAY
186517,7875.0,157500,157500.0,54000,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0.011,3,3,334026,-0.309,0.670,FRIDAY


In [20]:
pd.set_option('max_rows', 500)
pd.set_option('max_colwidth', 80)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 80]


In [21]:
data.columns

Index(['AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_INCOME_TOTAL',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_YEAR',
       'APPLICATION_DATE', 'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'CODE_GENDER',
       'DATE_DEFAULT_KNOWN', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
       'DAYS_LAST_PHONE_CHANGE', 'DAYS_REGISTRATION',
       'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DEFAULT',
       'EDUCATION_CLASS', 'ETHNICITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
       'EXT_SOURCE_3', 'FLAG_CONT_MOBILE', 'FLAG_DOCUMENT_10',
       'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13',
       'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16',
       'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19',
       'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21',
       'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4

In [22]:
data.shape

(311083, 80)

In [23]:

data.dtypes

AMT_ANNUITY                    float64
AMT_CREDIT                       int64
AMT_GOODS_PRICE                float64
AMT_INCOME_TOTAL                 int64
AMT_REQ_CREDIT_BUREAU_DAY      float64
AMT_REQ_CREDIT_BUREAU_HOUR     float64
AMT_REQ_CREDIT_BUREAU_MON      float64
AMT_REQ_CREDIT_BUREAU_QRT      float64
AMT_REQ_CREDIT_BUREAU_WEEK     float64
AMT_REQ_CREDIT_BUREAU_YEAR     float64
APPLICATION_DATE                object
CNT_CHILDREN                     int64
CNT_FAM_MEMBERS                float64
CODE_GENDER                     object
DATE_DEFAULT_KNOWN              object
DAYS_BIRTH                       int64
DAYS_EMPLOYED                    int64
DAYS_ID_PUBLISH                  int64
DAYS_LAST_PHONE_CHANGE         float64
DAYS_REGISTRATION                int64
DEF_30_CNT_SOCIAL_CIRCLE       float64
DEF_60_CNT_SOCIAL_CIRCLE       float64
DEFAULT                         object
EDUCATION_CLASS                  int64
ETHNICITY                        int64
EXT_SOURCE_1             

In [27]:
data.isna().sum().sort_values(axis=0, ascending=False)



DATE_DEFAULT_KNOWN             287238
OWN_CAR_AGE                    205314
EXT_SOURCE_1                   175408
HOUSETYPE_MODE                 156147
OCCUPATION_TYPE                 95599
EXT_SOURCE_3                    61729
AMT_REQ_CREDIT_BUREAU_DAY       42049
AMT_REQ_CREDIT_BUREAU_HOUR      42049
AMT_REQ_CREDIT_BUREAU_MON       42049
AMT_REQ_CREDIT_BUREAU_QRT       42049
AMT_REQ_CREDIT_BUREAU_WEEK      42049
AMT_REQ_CREDIT_BUREAU_YEAR      42049
DEFAULT                          2827
NAME_TYPE_SUITE                  1307
OBS_30_CNT_SOCIAL_CIRCLE         1032
OBS_60_CNT_SOCIAL_CIRCLE         1032
DEF_30_CNT_SOCIAL_CIRCLE         1032
DEF_60_CNT_SOCIAL_CIRCLE         1032
APPLICATION_DATE                  745
EXT_SOURCE_2                      667
SRC_VAR_4                         667
SRC_VAR_1                         667
AMT_GOODS_PRICE                   285
AMT_ANNUITY                        13
CNT_FAM_MEMBERS                     2
DAYS_LAST_PHONE_CHANGE              1
HOUR_APPR_PR